In [16]:
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import surprise
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from data_helpers import *

#Load train and testset for the surprise models
file_path = "../data/data_surprise.csv"
trainset, testset = build_surprise_data(file_path)

#Loads ratings to predict
INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

In [18]:
#Load train and testset for the custom models
train, test = split_data(load_data("../data/data_train.csv"), p_test = 0.1)

number of items: 10000, number of users: 1000


In [19]:
from implementations import *

In [20]:
Xtest = []
Xids = []

#Generate predictions with every method
rmse, Xtest, Xids, preds_test_kbm, preds_ids_kbm = knn_baseline_movie(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_kbu, preds_ids_kbu = knn_baseline_user(trainset, testset, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_svd, preds_ids_svd = svd(trainset, testset, ids, Xtest, Xids)

kNN Baseline Movie
   Training RMSE:  0.6241475383015472
   Test RMSE:  0.9908785231084433
kNN Baseline User
   Training RMSE:  0.6641531098089988
   Test RMSE:  0.9947164198559775
SVD
   Training RMSE:  0.9543999695353427
   Test RMSE:  1.0019284327486202


In [ ]:
rmse, Xtest, Xids, preds_test_als, preds_ids_als = matrix_factorization_als(train, test, ids, Xtest, Xids)
rmse, Xtest, Xids, preds_test_sgd, preds_ids_sgd = matrix_factorization_sgd(train, test, ids, Xtest, Xids)

start the ALS algorithm...
Train RMSE: 1.0761126862706574.
Train RMSE: 1.0339536582817281.
Train RMSE: 0.9964605772976493.
Train RMSE: 0.9671936482485406.
Train RMSE: 0.9483643759850089.
Train RMSE: 0.9366811926862457.
Train RMSE: 0.929188904791845.
Train RMSE: 0.9240717258734906.
Train RMSE: 0.9203721251135677.
Train RMSE: 0.9175826671586643.
Train RMSE: 0.9154155235907868.
Train RMSE: 0.9136943140543229.
Train RMSE: 0.9123037032959387.
Train RMSE: 0.9111644467269393.
Train RMSE: 0.9102200630975742.
Train RMSE: 0.9094291815786107.
Train RMSE: 0.9087608572388718.


In [1]:
np.array(Xtest) , np.array(Xids)

NameError: name 'np' is not defined

In [54]:
from sklearn.linear_model import LinearRegression
def blend(preds_test, preds_ids, testset):
    """
    Linear regression that finds the optimal weights of each model
    Argument : preds_test, predicted ratings for the known test set
               preds_ids, predicted ratings for the unknown set
               testset, the testset
    Return : estimations, the final predictions
             weights, coefficients associated to each model
    """
    print('Blending')
    
    #Known ratings of testset
    y_test = [rating for (_,_,rating) in testset]
    
    #Ridge Regression
    linreg = Ridge(alpha=0.0, normalize=True)
    
    #Fit between predicted and know ratings of testset
    linreg.fit(preds_test.T, y_test)
    weights = linreg.coef_
    
    #Predict unknown ratings
    predictions = np.clip(linreg.predict(preds_ids.T), 1, 5)
    
    print(weights, end='\n\n')
    
    #RMSE of regression
    print('Test RMSE: %f' % calculate_rmse(y_test, linreg.predict(preds_test.T)))
    
    #Rounding-off predictions
    estimations = np.zeros(len(predictions))
    for j, pred in enumerate(predictions):
        estimations[j] = round(pred)
        
    return estimations, weights

In [55]:
#Blending
predictions, weights = blend(np.array(Xtest), np.array(Xids), testset)

Blending
[-1.8606002   0.50975616  0.99641749 -0.63334515  1.14645031  0.74535184
 -0.12279898  0.26282306  0.00458331]

Test RMSE: 0.986415


In [36]:
len(predictions)

1176952

In [37]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")

File submission.csv ready to be submitted !
